Python implementation of a causal topic modeling paper.

This program implements the following paper:
<blockquote>
    <p>Hyun Duk Kim, Malu Castellanos, Meichun Hsu, ChengXiang Zhai, Thomas Rietz, and Daniel Diermeier. 2013. Mining causal topics in text data: Iterative topic modeling with time series feedback. In Proceedings of the 22nd ACM international conference on information & knowledge management (CIKM 2013). ACM, New York, NY, USA, 885-890. DOI=10.1145/2505515.2505612</p>
</blockquote>

In [19]:
# Import libraries
import math
import numpy as np
import os
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.api import VAR
import shutil
import tarfile
import xml.etree.ElementTree as ET
from mp_functions import token_pipeline
import multiprocessing as mp

In [2]:
# Import data
pres_market = pd.read_csv("./data/PRES00_WTA.csv")
AAMRQ = pd.read_csv("./data/AAMRQ.csv")
AAPL = pd.read_csv("./data/AAPL.csv")

In [3]:
pres_market

,Date,Contract,Units,$Volume,LowPrice,HighPrice,AvgPrice,LastPrice
0,5/1/2000,Dem,224,112.043,0.490,0.550,0.500,0.550
1,5/1/2000,Reform,2,0.067,0.019,0.048,0.034,0.019
2,5/1/2000,Rep,116,57.95,0.488,0.501,0.500,0.500
3,5/2/2000,Dem,87,44.369,0.501,0.522,0.510,0.508
4,5/2/2000,Reform,50,0.196,0.003,0.005,0.004,0.003
...,...,...,...,...,...,...,...,...
571,11/9/2000,Reform,"2,065",2.062,0.000,0.001,0.001,0.000
572,11/9/2000,Rep,"10,055",542.973,0.025,0.109,0.054,0.050
573,11/10/2000,Dem,"3,454","3,328.02",0.950,0.980,0.964,0.969
574,11/10/2000,Reform,23,0.02,0.000,0.001,0.001,0.000


In [4]:
AAMRQ

,Date,Open,High,Low,Close,Adj Close,Volume
0,7/3/2000,26.63,26.63,26.00,26.13,26.13,483100
1,7/5/2000,27.25,28.88,27.06,28.38,28.38,1840000
2,7/6/2000,28.44,29.56,27.81,29.00,29.00,1820000
3,7/7/2000,29.81,29.94,29.13,29.13,29.13,1150000
4,7/10/2000,29.75,30.13,29.19,30.00,30.00,711800
...,...,...,...,...,...,...,...
368,12/24/2001,21.72,21.73,20.77,21.19,21.19,1350000
369,12/26/2001,21.37,21.74,21.18,21.57,21.57,938900
370,12/27/2001,21.35,21.79,21.20,21.50,21.50,1190000
371,12/28/2001,21.60,22.19,21.55,22.00,22.00,853000


In [5]:
AAPL

,Date,Open,High,Low,Close,Adj Close,Volume
0,7/3/2000,0.930804,0.969866,0.930804,0.952009,0.821251,70828800
1,7/5/2000,0.950893,0.985491,0.906250,0.921875,0.795256,265216000
2,7/6/2000,0.937500,0.945313,0.886161,0.925223,0.798145,309545600
3,7/7/2000,0.939174,0.978795,0.930804,0.972098,0.838581,263603200
4,7/10/2000,0.965960,1.040179,0.959821,1.020089,0.879981,397796000
...,...,...,...,...,...,...,...
368,12/21/2001,0.375179,0.384643,0.371429,0.375000,0.323494,256334400
369,12/24/2001,0.373214,0.383036,0.373214,0.381429,0.329040,50629600
370,12/26/2001,0.381250,0.398214,0.377500,0.383750,0.331042,146400800
371,12/27/2001,0.385357,0.397321,0.385357,0.394107,0.339977,191508800


# Granger Test

"Granger tests...measur[e] statistical significance at different time lags using auto regression to identify causal relationships. Let $y_{t}$ and $x_{t}$ be two time series. To see if $x_{t}$ 'Granger causes' $y_{t}$ with maximum $p$ time lag, run the following regression:

$$
y_{t} = a_{0} + a_{1}y_{t−1} + ... + a_{p}y_{t−p} + b_{1}x_{t−1} + ... + b_{p}x_{t−p}
$$

Then, use F-tests to evaluate the significance of the lagged $x$ terms. The coefficients of lagged $x$ terms estimate the impact of $x$ on $y$. We average the $x$ term coefficients, $\frac{\sum_{i=1}^{p}b_{i}}{|b|}$, as an impact value."

In [30]:
close = pd.concat([AAMRQ["Close"], AAPL["Close"]], axis=1, keys=["AAMRQ", "AAPL"])
close = close.rolling(3, center=True, min_periods=2).mean()
close = close.diff()[1:]
close

,AAMRQ,AAPL
1,0.581667,-0.003906
2,1.000000,0.006696
3,0.540000,0.032738
4,0.146667,0.030506
5,0.500000,0.026414
...,...,...
368,-0.026667,-0.001547
369,-0.130000,0.004881
370,0.270000,0.006369
371,0.243333,0.006369


In [9]:
# Is first column "caused by" second column up to a given lag?
gc_res = grangercausalitytests(close, 5)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.4416  , p=0.5068  , df_denom=368, df_num=1
ssr based chi2 test:   chi2=0.4452  , p=0.5046  , df=1
likelihood ratio test: chi2=0.4449  , p=0.5048  , df=1
parameter F test:         F=0.4416  , p=0.5068  , df_denom=368, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.5103  , p=0.6008  , df_denom=365, df_num=2
ssr based chi2 test:   chi2=1.0345  , p=0.5961  , df=2
likelihood ratio test: chi2=1.0331  , p=0.5966  , df=2
parameter F test:         F=0.5103  , p=0.6008  , df_denom=365, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.3828  , p=0.7655  , df_denom=362, df_num=3
ssr based chi2 test:   chi2=1.1706  , p=0.7601  , df=3
likelihood ratio test: chi2=1.1687  , p=0.7605  , df=3
parameter F test:         F=0.3828  , p=0.7655  , df_denom=362, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=1.0379  , p=0.3875  

In [10]:
p_vals = []
for i in range(1, len(gc_res) + 1):
    p_vals.append(gc_res[i][0]['params_ftest'][1])

In [11]:
p_vals

[0.5067873616230052,
 0.6007552098554052,
 0.7654686970619549,
 0.3874795496740359,
 0.4398020489287572]

In [12]:
np.argmin(p_vals)

3

In [13]:
np.argmax(np.subtract(1, p_vals))

3

In [213]:
sig = np.subtract(1, p_vals)

In [217]:
sig

array([0.49321264, 0.39924479, 0.2345313 , 0.61252045, 0.56019795])

In [14]:
gc_res[1][1][0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.414
Model:                            OLS   Adj. R-squared:                  0.413
Method:                 Least Squares   F-statistic:                     260.8
Date:                Mon, 23 Nov 2020   Prob (F-statistic):           9.31e-45
Time:                        20:37:28   Log-Likelihood:                -233.09
No. Observations:                 371   AIC:                             470.2
Df Residuals:                     369   BIC:                             478.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.6428      0.040     16.150      0.000       0.565       0.721
const         -0.0061      0.024     -0.259      0.796      -0.053       0.040
==============================================================================
Omnibus:                      161.879   Durbin-Watson:                   1.785
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3606.831
Skew:                          -1.284   Prob(JB):                         0.00
Kurtosis:                      18.058   Cond. No.                         1.69
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
model = VAR(close)
results = model.fit(5)
results.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 23, Nov, 2020
Time:                     20:37:28
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -10.2797
Nobs:                     367.000    HQIC:                  -10.4208
Log likelihood:           909.784    FPE:                2.71597e-05
AIC:                     -10.5138    Det(Omega_mle):     2.56020e-05
--------------------------------------------------------------------
Results for equation AAMRQ
              coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------
const           -0.009421         0.021853           -0.431           0.666
L1.AAMRQ         0.800274         0.052984           15.104           0.000
L1.AAPL          1.023943         1.797251            0.570           0.569
L2.A

In [16]:
def impact_value(data, lag):
    model = VAR(data)
    results = model.fit(lag)
    numerator = 0
    for i in range(1, lag + 1):
        numerator += results.params[results.params.columns.values[0]]['L' + str(i) + '.' + results.params.columns.values[1]]
    return numerator / np.abs(lag)

In [220]:
impact_value(close, 1)

-0.8733952397899155

# Topic Purity

In [17]:
def topic_purity(num_pos_impact, num_neg_impact, num_significant):
    p_prob = num_pos_impact / num_significant
    n_prob = num_neg_impact / num_significa
    entropy = p_prob * np.log(p_prob) + n_prob * np.log(n_prob)
    return 100 + 100 * entropy

# Import NYTAC

Do not run this code below. It was to run the original data cleaning steps. Running again will delete the NYTAC in storage. It has been commented out for safety.

In [18]:
# tars = []
# for root, dirs, files in os.walk("./data/nyt_corpus/data"):
#     if dirs:
#         delete = dirs.copy()
#         delete[:] = [x for x in dirs if x not in ['2000', '2001', '2002', '2003']]
#         dirs[:] = [x for x in dirs if x in ['2000', '2001', '2002', '2003']]
#         for name in delete:
#             subdir = os.path.join(root, name)
#             with os.scandir(subdir) as it:
#                 for entry in it:
#                     os.remove(entry)
#             os.rmdir(subdir)
#     if files:
#         if os.path.basename(root) == '2003':
#             delete = files.copy()
#             delete = [x for x in files if x not in ['01.tgz', '02.tgz', '03.tgz']]
#             files[:] = [x for x in files if x in ['01.tgz', '02.tgz', '03.tgz']]
#             for name in delete:
#                 os.remove(os.path.join(root, name))
#         for file in files:
#             tars.append(os.path.join(root, file))

# for file_path in tars:
#     tar = tarfile.open(file_path)
#     tar.extractall(path=os.path.dirname(file_path))
#     tar.close()
#     os.remove(file_path)

# # collect articles for 2000 Presidential Election
# with os.scandir("./data/nyt_corpus/data/2000") as it:
#     for entry in it:
#         if os.path.basename(entry) in ['05', '06', '07', '08', '09', '10']:
#             shutil.copytree(entry, os.path.join("./data/nyt_corpus/data/election/2000", os.path.basename(entry)))

# # collect articles for Stock Time Series, AAMRQ vs. AAPL
# with os.scandir("./data/nyt_corpus/data/2000") as it:
#     for entry in it:
#         if os.path.basename(entry) in ['07', '08', '09', '10', '11', '12']:
#             shutil.copytree(entry, os.path.join("./data/nyt_corpus/data/stock/2000", os.path.basename(entry)))
# with os.scandir("./data/nyt_corpus/data/2001") as it:
#     for entry in it:
#         shutil.copytree(entry, os.path.join("./data/nyt_corpus/data/stock/2001", os.path.basename(entry)))

# # collect articles for Iraq War
# with os.scandir("./data/nyt_corpus/data/2002") as it:
#     for entry in it:
#         shutil.copytree(entry, os.path.join("./data/nyt_corpus/data/war/2002", os.path.basename(entry)))
# with os.scandir("./data/nyt_corpus/data/2003") as it:
#     for entry in it:
#         if os.path.basename(entry) in ['01', '02', '03']:
#             shutil.copytree(entry, os.path.join("./data/nyt_corpus/data/war/2003", os.path.basename(entry)))

# # remove unused directories
# for year in ['2000', '2001', '2002', '2003']:
#     shutil.rmtree(os.path.join("./data/nyt_corpus/data", os.path.basename(year)))

# # initialize list of documents to delete
# delete = []

# # delete documents that do not contain "Bush" and "Gore" or do not contain document bodies
# for root, dirs, files in os.walk("./data/nyt_corpus/data/election"):
#     if files:
#         for name in files:
#             tree = ET.parse(os.path.join(root, name))
#             tree_root = tree.getroot()
#             element = tree_root.find('./body/body.content/block[@class="full_text"]')
#             if element:
#                 keep = 0
#                 Bush = 0
#                 Gore = 0
#                 for para in element.findall('p'):
#                     para_list = nltk.word_tokenize(para.text)
#                     if 'Bush' in para_list:
#                         Bush = 1
#                     if 'Gore' in para_list:
#                         Gore = 1
#                     keep = Bush * Gore
#                 if not keep:
#                     delete.append(os.path.join(root, name))
#             else:
#                 delete.append(os.path.join(root, name))

# # delete documents that do not contain document bodies
# for root, dirs, files in os.walk("./data/nyt_corpus/data/stock"):
#     if files:
#         for name in files:
#             tree = ET.parse(os.path.join(root, name))
#             tree_root = tree.getroot()
#             element = tree_root.find('./body/body.content/block[@class="full_text"]')
#             if not element:
#                 delete.append(os.path.join(root, name))

# # delete documents that do not contain "Iraq" or do not contain document bodies
# for root, dirs, files in os.walk("./data/nyt_corpus/data/war"):
#     if files:
#         for name in files:
#             tree = ET.parse(os.path.join(root, name))
#             tree_root = tree.getroot()
#             element = tree_root.find('./body/body.content/block[@class="full_text"]')
#             if element:
#                 keep = 0
#                 for para in element.findall('p'):
#                     para_list = nltk.word_tokenize(para.text)
#                     if 'Iraq' in para_list:
#                         keep = 1
#                 if not keep:
#                     delete.append(os.path.join(root, name))
#             else:
#                 delete.append(os.path.join(root, name))

# # delete the unneeded documents
# for name in delete:
#     os.remove(name)

# # delete empty directories
# for root, dirs, files in os.walk("./data/nyt_corpus/data"):
#     if not dirs and not files:
#         os.rmdir(root)

# # consolidate xml files into text files
# # one text file contains the documents from the date in the file name
# # documents are stored one document per line
# for root, dirs, files in os.walk("./data/nyt_corpus/data"):
#     if files:
#         yyyy = os.path.basename(os.path.dirname(os.path.dirname(root)))
#         mm = os.path.basename(os.path.dirname(root))
#         dd = os.path.basename(root)
#         file_name = yyyy + "-" + mm + "-" + dd + ".txt"
#         base_folder = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(root))))
#         directory = os.path.join("./data/nyt_corpus/data", base_folder)
#         f = open(os.path.join(directory, file_name), "w")
#         for name in files:
#             tree = ET.parse(os.path.join(root, name))
#             tree_root = tree.getroot()
#             element = tree_root.find('./body/body.content/block[@class="full_text"]')
#             paragraphs = []
#             for para in element.findall('p'):
#                 paragraphs.append(para.text)
#             f.write(" ".join(paragraphs) + "\n")
#         f.close()

# # remove unused directories
# for subdir in ["election/2000", "stock/2000", "stock/2001", "war/2002", "war/2003"]:
#     shutil.rmtree(os.path.join("./data/nyt_corpus/data", subdir))

# Set Up Corpus

In [95]:
(np.datetime64('2018-02-01') - np.datetime64('2018-01-01')).astype(np.int64)

31

In [20]:
# def build_corpus(self):
#     """
#     Read document, fill in self.documents, a list of list of word
#     self.documents = [["the", "day", "is", "nice", "the", ...], [], []...]

#     Update self.number_of_documents
#     """
#     with os.scandir(self.documents_path) as it:
#         s0 = time.time()
#         for entry in it:
#             e0 = time.time()
#             print("time_0: {}".format(e0 - s0))
#             s0 = time.time()
#             with open(entry, 'r') as file:
#                 s1 = time.time()
#                 while True:
#                     e1 = time.time()
#                     print("time_1: {}".format(e1 - s1))
#                     s1 = time.time()
#                     line = file.readline()
#                     if not line:
#                         break
#                     tokens = word_tokenize(line.lower())
#                     tokens= [x for x in tokens if x.isalnum()]
#                     nltk_stop_words = nltk.corpus.stopwords.words('english')
#                     tokens = [x for x in tokens if x not in nltk_stop_words]
#                     tokens = normalise(tokens, variety="AmE", verbose = False)
#                     tokens = [WordNetLemmatizer().lemmatize(word, pos='v') for word in tokens]
#                     tokens = [WordNetLemmatizer().lemmatize(word, pos='n') for word in tokens]
#                     self.documents.append(tokens)
#                     self.doc_timestamps.append(os.path.basename(entry).split(".")[0])
#                     self.number_of_documents += 1

In [20]:
def normalize(input_matrix):
    """
    Normalizes the rows of a 2d input_matrix so they sum to 1
    """

    row_sums = input_matrix.sum(axis=1)
    try:
        assert (np.count_nonzero(row_sums)==np.shape(row_sums)[0]) # no row should sum to zero
    except Exception:
        raise Exception("Error while normalizing. Row(s) sum to zero")
    new_matrix = input_matrix / row_sums[:, np.newaxis]
    return new_matrix

class Corpus(object):

    """
    A collection of documents.
    """

    def __init__(self, documents_path):
        """
        Initialize empty document list.
        """
        self.documents = []
        self.doc_timestamps = []
        self.vocabulary = []
        self.likelihoods = []
        self.documents_path = documents_path
        self.term_doc_matrix = None
        self.document_topic_prob = None  # P(z | d)
        self.topic_word_prob = None  # P(w | z)
        self.topic_prob = None  # P(z | d, w)

        self.number_of_documents = 0
        self.vocabulary_size = 0

    def build_corpus(self):
        """
        Read document, fill in self.documents, a list of list of word
        self.documents = [["the", "day", "is", "nice", "the", ...], [], []...]

        Update self.number_of_documents
        """
        lines = []
        with os.scandir(self.documents_path) as it:
            for entry in it:
                with open(entry, 'r') as file:
                    while True:
                        line = file.readline()
                        if not line:
                            break
                        lines.append(line)
                        self.doc_timestamps.append(os.path.basename(entry).split(".")[0])
                        self.number_of_documents += 1
        with mp.Pool() as pool:
            self.documents = pool.map(token_pipeline, lines)

    def build_vocabulary(self):
        """
        Construct a list of unique words in the whole corpus. Put it in self.vocabulary
        for example: ["rain", "the", ...]

        Update self.vocabulary_size
        """
        word_set = set()
        for document in self.documents:
            for word in document:
                word_set.add(word)
        self.vocabulary = list(word_set)
        self.vocabulary_size = len(self.vocabulary)

    def build_term_doc_matrix(self):
        """
        Construct the term-document matrix where each row represents a document,
        and each column represents a vocabulary term.

        self.term_doc_matrix[i][j] is the count of term j in document i
        """
        self.term_doc_matrix = np.zeros((self.number_of_documents, self.vocabulary_size))
        for i, document in enumerate(self.documents):
            for j, word in enumerate(self.vocabulary):
                self.term_doc_matrix[i][j] = document.count(word)

    def initialize_randomly(self, number_of_topics):
        """
        Randomly initialize the matrices: document_topic_prob and topic_word_prob
        which hold the probability distributions for P(z | d) and P(w | z): self.document_topic_prob, and self.topic_word_prob

        Don't forget to normalize!
        HINT: you will find numpy's random matrix useful [https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.random.html]
        """
        self.document_topic_prob = np.random.random_sample((self.number_of_documents, number_of_topics))
        self.document_topic_prob = normalize(self.document_topic_prob) # P(z | d)

        self.topic_word_prob = np.random.random_sample((number_of_topics, len(self.vocabulary)))
        self.topic_word_prob = normalize(self.topic_word_prob) # P(w | z)

    def initialize_uniformly(self, number_of_topics):
        """
        Initializes the matrices: self.document_topic_prob and self.topic_word_prob with a uniform
        probability distribution. This is used for testing purposes.

        DO NOT CHANGE THIS FUNCTION
        """
        self.document_topic_prob = np.ones((self.number_of_documents, number_of_topics))
        self.document_topic_prob = normalize(self.document_topic_prob) # P(z | d)

        self.topic_word_prob = np.ones((number_of_topics, len(self.vocabulary)))
        self.topic_word_prob = normalize(self.topic_word_prob) # P(w | z)

    def initialize(self, number_of_topics, random=False):
        """ Call the functions to initialize the matrices document_topic_prob and topic_word_prob
        """
        print("Initializing...")

        if random:
            self.initialize_randomly(number_of_topics)
        else:
            self.initialize_uniformly(number_of_topics)

    def expectation_step(self):
        """ The E-step updates P(z | w, d)
        """
        print("E step:")

        for i in range(self.topic_prob.shape[1]):
            self.topic_prob[:, i, :] =  np.outer(self.document_topic_prob[:, i], self.topic_word_prob[i, :])
            self.topic_prob[:, i, :] /= np.matmul(self.document_topic_prob, self.topic_word_prob)

    def maximization_step(self, number_of_topics):
        """ The M-step updates P(w | z) and P(z | d)
        """
        print("M step:")

        # update P(w | z)
        for i in range(number_of_topics):
            self.topic_word_prob[i, :] = np.diag(np.matmul(np.transpose(self.term_doc_matrix), self.topic_prob[:, i, :]))
            self.topic_word_prob[i, :] /= np.sum(self.topic_word_prob[i, :])

        # update P(z | d)
        for i in range(number_of_topics):
            self.document_topic_prob[:, i] = np.diag(np.matmul(self.term_doc_matrix, np.transpose(self.topic_prob[:, i, :])))
        self.document_topic_prob /= np.sum(self.document_topic_prob, axis = 1)[:, None]

    def calculate_likelihood(self, number_of_topics):
        """ Calculate the current log-likelihood of the model using
        the model's updated probability matrices

        Append the calculated log-likelihood to self.likelihoods
        """
        self.likelihoods.append(np.sum(np.multiply(self.term_doc_matrix, np.log(np.matmul(self.document_topic_prob, self.topic_word_prob)))))
        return self.likelihoods[-1]

    def plsa(self, number_of_topics, max_iter, epsilon):

        """
        Model topics.
        """
        print ("EM iteration begins...")

        # build term-doc matrix
        self.build_term_doc_matrix()

        # Create the counter arrays.

        # P(z | d, w)
        self.topic_prob = np.zeros([self.number_of_documents, number_of_topics, self.vocabulary_size], dtype=np.float)

        # P(z | d) P(w | z)
        self.initialize(number_of_topics, random=True)

        # Run the EM algorithm
        current_likelihood = 0.0

        for iteration in range(max_iter):
            print("Iteration #" + str(iteration + 1) + "...")
            self.expectation_step()
            self.maximization_step(number_of_topics)
            previous_likelihood = current_likelihood
            current_likelihood = self.calculate_likelihood(number_of_topics)
            if np.abs(current_likelihood - previous_likelihood) < epsilon:
                break

In [21]:
corpus = Corpus("./data/nyt_corpus/data/test")

In [22]:
# start 5:41 PM, end 6:08 PM
# start 6:10 PM, end 6:37 PM
# start 8:38 PM, end 8:48 PM
corpus.build_corpus()

In [23]:
corpus.build_vocabulary()

In [26]:
# start 7:14 PM, end 8:08 PM
# start 8:51 PM, end 9:05 PM
number_of_topics = 2
max_iterations = 50
epsilon = 0.001
corpus.plsa(number_of_topics, max_iterations, epsilon)

EM iteration begins...
Initializing...
Iteration #1...
E step:
M step:
Iteration #2...
E step:
M step:
Iteration #3...
E step:
M step:
Iteration #4...
E step:
M step:
Iteration #5...
E step:
M step:
Iteration #6...
E step:
M step:
Iteration #7...
E step:
M step:
Iteration #8...
E step:
M step:
Iteration #9...
E step:
M step:
Iteration #10...
E step:
M step:
Iteration #11...
E step:
M step:
Iteration #12...
E step:
M step:
Iteration #13...
E step:
M step:
Iteration #14...
E step:
M step:
Iteration #15...
E step:
M step:
Iteration #16...
E step:
M step:
Iteration #17...
E step:
M step:
Iteration #18...
E step:
M step:
Iteration #19...
E step:
M step:
Iteration #20...
E step:
M step:
Iteration #21...
E step:
M step:
Iteration #22...
E step:
M step:
Iteration #23...
E step:
M step:
Iteration #24...
E step:
M step:
Iteration #25...
E step:
M step:
Iteration #26...
E step:
M step:
Iteration #27...
E step:
M step:
Iteration #28...
E step:
M step:
Iteration #29...
E step:
M step:
Iteration #30

In [128]:
d = pd.Series(corpus.doc_timestamps).astype("datetime64")[pd.Series(corpus.doc_timestamps).astype("datetime64") == AAPL["Date"].astype("datetime64")[50]].index

In [198]:
df.append(corpus.document_topic_prob[d].sum(axis=0))

In [208]:
df = np.stack(df, axis = 1)

In [212]:
df.shape[0]

2

In [191]:
df = np.append(df,corpus.document_topic_prob[d].sum(axis=0), axis=0)

In [107]:
AAPL["Date"].astype("datetime64")[50]

Timestamp('2000-09-13 00:00:00')

In [102]:
AAPL["Date"].astype("datetime64")[AAPL["Date"].astype("datetime64") == pd.Series(corpus.doc_timestamps).astype("datetime64")[0]].index[0]

43

In [ ]:
def topic_level_causality(document_topic_prob, doc_timestamps, time_series, ts_timestamps, tn):
    time_series_sub = []
    ts = []
    for k, t in enumerate(ts_timestamps):
        doc_i = doc_timestamps[doc_timestamps == t].index
        if doc_i.any():
            time_series_sub.append(time_series.iloc[k])
            ts.append(document_topic_prob[doc_i].sum(axis = 0))
    ts = np.stack(ts, axis = 1)
    for topic in range(ts.shape[0]):
        gc_res = grangercausalitytests(close, 5)
        p_vals = []
        for i in range(1, len(gc_res) + 1):
            p_vals.append(gc_res[i][0]['params_ftest'][1])
    return

In [218]:
corpus.vocabulary

['meet',
 'shetland',
 'perennially',
 'missionary',
 'valedictory',
 'concerto',
 'enumerate',
 'mandate',
 'barnstorm',
 'delgado',
 'posterity',
 'peanut',
 'elaboration',
 'lamentable',
 'ocher',
 'questionable',
 'pretext',
 'bennett',
 'calista',
 'patella',
 'result',
 'arlington',
 'crawford',
 'elicit',
 'extraordinary',
 'overblown',
 'carbine',
 'enunciate',
 'wilma',
 'beam',
 'snigger',
 'cowardly',
 'mindy',
 'recession',
 'centre',
 'stuyvesant',
 'serrano',
 'chapin',
 'customize',
 'decisively',
 'outlast',
 'tenderloin',
 'analogous',
 'bourbon',
 'bogged',
 'engender',
 'meal',
 'supagroup',
 'unremitting',
 'omit',
 'trombonist',
 'snakeskin',
 'helm',
 'gallon',
 'desouza',
 'distinction',
 'coney',
 'lockheed',
 'ibrahim',
 'weariness',
 'wont',
 'velazquez',
 'zedillo',
 'biogen',
 'vengeance',
 'taint',
 'fringe',
 'heap',
 'cryptic',
 'crewman',
 'E twenty four',
 'exceptionally',
 'yogi',
 'auchincloss',
 'sideline',
 'nineteen sixteen',
 'clergyman',
 'schako

In [52]:
np.sum(corpus.document_topic_prob, axis=1) # P(z | d) [num documents, num topics]

array([1., 1., 1., ..., 1., 1., 1.])

In [50]:
corpus.topic_word_prob # P(w | z)  [num topics, num words in vocabulary]

array([1., 1.])

In [47]:
corpus.topic_prob # P(z | d, w) [num documents, num topics, num words in vocabulary]

array([[[9.72766790e-01, 1.00000000e+00, 7.71679371e-01, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
        [2.72332101e-02, 7.51023205e-53, 2.28320629e-01, ...,
         3.56863371e-47, 2.70435982e-29, 6.96381040e-32]],

       [[9.68082447e-01, 1.00000000e+00, 7.41594437e-01, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
        [3.19175526e-02, 8.84464672e-53, 2.58405563e-01, ...,
         4.20270695e-47, 3.18486927e-29, 8.20113711e-32]],

       [[9.19375996e-01, 1.00000000e+00, 5.18993308e-01, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
        [8.06240038e-02, 2.35252613e-52, 4.81006692e-01, ...,
         1.11784882e-46, 8.47121248e-29, 2.18136348e-31]],

       ...,

       [[4.05182041e-01, 1.00000000e+00, 6.05509797e-02, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
        [5.94817959e-01, 3.93819429e-51, 9.39449020e-01, ...,
         1.87131007e-45, 1.41810457e-27, 3.65166325e-30]],

       [[2.33270459